In [1]:
# === Funções utilitárias para TIF -> GeoJSON ===
import math, json, time
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import Affine
from rasterio.features import shapes as raster_shapes
from shapely.geometry import shape as shp_shape, mapping as shp_mapping, Polygon, MultiPolygon
from shapely.ops import unary_union
from pyproj import Geod, Transformer

def infer_threshold_native(src, threshold):
    """Infere escala nativa (0..1, 0..100, 0..10000) a partir de uma amostra."""
    print("Inferring native threshold scale...")
    first_window = None
    for _, w in src.block_windows(1):
        first_window = w
        break
    if first_window is None:
        return threshold, 1.0, None
    arr = src.read(1, window=first_window, masked=True)
    valid = arr.compressed() if np.ma.isMaskedArray(arr) else arr.flatten()
    if valid.size == 0:
        return threshold, 1.0, None
    data_max = float(np.nanpercentile(valid, 99.9))
    if data_max <= 1.0: scale = 1.0
    elif data_max <= 110.0: scale = 100.0
    elif data_max <= 11000.0: scale = 10000.0
    else: scale = 1.0
    thr_native = threshold * scale
    return thr_native, scale, data_max

def read_array_with_overview(src, overview=0):
    print(f"Reading array with overview={overview}...")
    if overview and overview > 1:
        new_h = max(1, src.height // overview)
        new_w = max(1, src.width  // overview)
        arr = src.read(1, out_shape=(new_h, new_w), resampling=Resampling.nearest)
        # ajustar transform para refletir o downsample
        scale_x = src.width / float(new_w)
        scale_y = src.height / float(new_h)
        new_transform = src.transform * Affine.scale(scale_x, scale_y)
        return arr, new_transform
    else:
        print("Reading full resolution array...")
        arr = src.read(1)
        return arr, src.transform

def apply_bbox_mask_bool(mask_bool, transform, bbox):
    if bbox is None:
        print("No bbox provided, skipping bbox masking.")
        return mask_bool
    
    print(f"Applying bbox mask: {bbox}")
    minx, miny, maxx, maxy = bbox
    inv = ~transform
    # Quatro cantos do bbox -> índices aproximados
    cr = [inv * (minx, miny), inv * (minx, maxy), inv * (maxx, miny), inv * (maxx, maxy)]
    cols = [c for (c, r) in cr]; rows = [r for (c, r) in cr]
    rmin = int(max(0, math.floor(min(rows)))); rmax = int(min(mask_bool.shape[0], math.ceil(max(rows))))
    cmin = int(max(0, math.floor(min(cols)))); cmax = int(min(mask_bool.shape[1], math.ceil(max(cols))))
    out = np.zeros_like(mask_bool, dtype=bool)
    out[rmin:rmax, cmin:cmax] = mask_bool[rmin:rmax, cmin:cmax]
    return out

def geodesic_area_m2(geom):
    print("Calculating geodesic area...")
    geod = Geod(ellps="WGS84")
    
    def poly_area(p: Polygon):
        x, y = p.exterior.coords.xy
        area, _ = geod.polygon_area_perimeter(x, y)[:2]
        return abs(area)
    
    if isinstance(geom, Polygon):
        print("Calculating area for a single Polygon...")
        return poly_area(geom)
    elif isinstance(geom, MultiPolygon):
        print("Calculating area for a MultiPolygon...")
        return sum(poly_area(g) for g in geom.geoms)
    
    return 0.0

def ensure_wgs84(geom, src_crs):
    print("Ensuring geometry is in WGS84...")
    if src_crs is None or src_crs.to_string() == "EPSG:4326":
        print("Source CRS is already EPSG:4326, no transformation needed.")
        return geom
    
    transformer = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)

    def _proj(coords):
        xs, ys = zip(*coords)
        X, Y = transformer.transform(xs, ys)
        return list(zip(X, Y))
    
    if isinstance(geom, Polygon):
        print("Transforming a single Polygon...")
        ext = _proj(list(geom.exterior.coords))
        holes = [_proj(list(r.coords)) for r in geom.interiors]
        return Polygon(ext, holes)
    elif isinstance(geom, MultiPolygon):
        print("Transforming a MultiPolygon...")
        return MultiPolygon([ensure_wgs84(g, src_crs) for g in geom.geoms])
    return geom

# === Configurações para salvar em GeoJSON ===
# "outputs/tropical_tree_cover_data/10S_050W_F10m.tif", 
# "outputs/tropical_tree_cover_data/20S_060W_F10m.tif"
TIF_PATHS = [
    "outputs/tropical_tree_cover_data/20S_050W_F10m.tif",
    "outputs/tropical_tree_cover_data/10S_050W_F10m.tif"    
]

OUT_GEOJSON = None  # Se None, salva ao lado do TIF com sufixo .metadata.json

# Modo contínuo (probabilidade)
USE_THRESHOLD = True
THRESHOLD = 0.4     # 40%
AUTOSCALE = True    # tenta detectar 0..1, 0..100 ou 0..10000

# OU modo binário/categórico (use USE_THRESHOLD=False)
CLASS_VALUE = 1     # valor a poligonizar (ex.: 1)

# Controles de complexidade
OVERVIEW = 8        # 0=full; 2/4/8 para reduzir a resolução antes de poligonizar
SIMPLIFY = 0.0      # tolerância de simplificação (no CRS do raster; se EPSG:4326, é em graus)
MIN_AREA_M2 = 0.0   # remove polígonos com área < X m² (área geodésica WGS84)
DISSOLVE = False    # True para unir tudo em um MultiPolygon

# Recorte opcional por bbox no CRS do raster: (minx, miny, maxx, maxy) ou None
BBOX = None  # exemplo: (-50.0, -20.0, -49.5, -19.5)

print("Input:", TIF_PATHS)
print("Output:", OUT_GEOJSON or "(auto ao lado do TIF)")

Input: ['outputs/tropical_tree_cover_data/20S_050W_F10m.tif', 'outputs/tropical_tree_cover_data/10S_050W_F10m.tif']
Output: (auto ao lado do TIF)


In [ ]:
# === Execução: TIF -> GeoJSON ===
from pathlib import Path

for TIF_PATH in TIF_PATHS:
    t0 = time.time()
    tif_path = Path(TIF_PATH)
    assert tif_path.exists(), f"Arquivo não encontrado: {tif_path}"
    OUT_GEOJSON = TIF_PATH.replace(".tif", ".geojson")
    out_path = Path(OUT_GEOJSON)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"\nProcessando: {tif_path}  ->  {out_path}")

    with rasterio.open(tif_path) as src:
        print(f"Using rasterio to open the tif file: {tif_path}.")
        arr, transform = read_array_with_overview(src, OVERVIEW)

        print("Gerar máscara: threshold (contínuo) OU valor (binário/categórico)")
        if USE_THRESHOLD:
            thr = THRESHOLD
            if AUTOSCALE:
                thr_native, scale, data_max = infer_threshold_native(src, thr)
            else:
                thr_native = thr
            mask = (arr >= thr_native)
        else:
            val = CLASS_VALUE
            mask = (arr == val) if float(val).is_integer() else np.isclose(arr, val)

        print("Recorte por bbox (opcional)")
        mask = mask.astype(bool) #apply_bbox_mask_bool(mask.astype(bool), transform, BBOX)

        print("Extrai polígonos para pixels=1")
        mask_uint8 = mask.astype(np.uint8)
        geoms = []
        for geom, value in raster_shapes(mask_uint8, mask=None, transform=transform, connectivity=4):
            if value != 1:
                continue
            geoms.append(shp_shape(geom))

        print("Simplify")
        if SIMPLIFY and SIMPLIFY > 0:
            geoms = [g.simplify(SIMPLIFY, preserve_topology=True) for g in geoms]

        print("Filtro por área geodésica mínima")
        if MIN_AREA_M2 and MIN_AREA_M2 > 0:
            filtered = []
            for g in geoms:
                gw = ensure_wgs84(g, src.crs)
                if geodesic_area_m2(gw) >= MIN_AREA_M2:
                    filtered.append(g)
            geoms = filtered

        print("Dissolve")
        if DISSOLVE and geoms:
            d = unary_union(geoms)
            if isinstance(d, (Polygon, MultiPolygon)):
                geoms = [d]
            else:
                geoms = []

    print("Monta FeatureCollection e salva")
    features = [{
        "type": "Feature",
        "properties": {
            "source": tif_path.name,
            "mode": "threshold" if USE_THRESHOLD else "class",
            "threshold": THRESHOLD if USE_THRESHOLD else None,
            "autoscale": bool(AUTOSCALE) if USE_THRESHOLD else None,
            "class_value": None if USE_THRESHOLD else CLASS_VALUE,
            "overview": OVERVIEW,
            "simplify": SIMPLIFY,
            "min_area_m2": MIN_AREA_M2,
            "dissolve": bool(DISSOLVE),
        },
        "geometry": shp_mapping(g)
    } for g in geoms]

    fc = {"type": "FeatureCollection", "features": features}
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(fc, f, ensure_ascii=False)

    elapsed = time.time() - t0
    print(f"GeoJSON salvo em: {out_path}  | features: {len(features)}  | tempo: {elapsed:.1f}s")


Processando: outputs\tropical_tree_cover_data\20S_050W_F10m.tif  ->  outputs\tropical_tree_cover_data\20S_050W_F10m.geojson
Using rasterio to open the tif file: outputs\tropical_tree_cover_data\20S_050W_F10m.tif.
Reading array with overview=8...
Gerar máscara: threshold (contínuo) OU valor (binário/categórico)
Inferring native threshold scale...
Recorte por bbox (opcional)
Extrai polígonos para pixels=1
